In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

# Define a function to interpolate missing time steps in a DataFrame 
def interpolate_missing_time_steps(df):
    time_column = df.columns[0]  # Extract the time column name
    probability_columns = df.columns[1:]  # Extract the names of the probability columns
    df[time_column] = df[time_column].astype(int)  # Convert time values to integers
    full_time_range = range(df[time_column].min(), df[time_column].max() + 1)  # Generate the full range of time steps
    df_reindexed = df.set_index(time_column).reindex(full_time_range).reset_index()  # Reindex the DataFrame to fill missing time steps
    df_reindexed[probability_columns] = df_reindexed[probability_columns].interpolate()  # Interpolate missing probability values
    return df_reindexed  # Return the reindexed DataFrame

def lcr(df, k=1):
    probabilities = df.iloc[:, 1:].values  # Exclude the 'Time' column for calculations
    strengths = np.exp(k * probabilities)

    # Sum strengths per row
    sum_strengths = strengths.sum(axis=1)

    # Compute Luce choice probabilities
    luce_probs = strengths / sum_strengths[:, np.newaxis]
    
    # Step 2: Compute scaling factor Δ_t for each row
    # max(act(t)): maximum activation in the current row
    row_max = df.iloc[:, 1:].max(axis=1)
    # max(act(overall)): global maximum activation across all rows and all items
    global_max = df.iloc[:, 1:].max().max()
    delta = row_max / global_max

    # Step 3: Compute final fixation probabilities p(R_i) = Δ_t * L_i
    # Convert delta to numpy array before reshaping to avoid pandas multi-dimensional indexing error
    fixation_probs = luce_probs * delta.values[:, np.newaxis]

    transformed_df = df.copy()  # Create a copy of the original DataFrame to store results
    transformed_df.iloc[:, 1:] = fixation_probs  # Replace the original probabilities with the fixation probabilities
    return transformed_df

# Define a function to read a CSV file, preprocess data, and apply optional transformations
def read_csv(file_path, shift=False, apply_softmax=False, tmult=1, C=5, k=1, 
             generate_cross=True, apply_lcr_first=False, apply_lcr=False, 
             normalize=False, rescale=False, rescale_first=False, do_interpolate_missing_time_steps=True,
             bgate=False, bthresh=0.01, tmax=1000, sum_weight=1):
    df = pd.read_csv(file_path)  # Read the CSV file into a DataFrame
    probability_columns = df.columns[1:]  # Extract the names of the probability columns

    if bgate:  # Apply bgate logic if bgate is True
        # print(f'Applying bgate with threshold {threshold}')
        row_sums = df[probability_columns].sum(axis=1)  # Sum probabilities across columns for each row
        mask = row_sums < bthresh  # Identify rows where the sum of probabilities is below the threshold
        df.loc[mask, probability_columns] = 0  # Set probabilities to 0 for those rows
    
    if rescale_first:  # Apply rescaling if rescale is True
        print('Rescaling data first')
        df_min = df[probability_columns].min().min()  # Find the minimum value across all columns
        df_max = df[probability_columns].max().max()  # Find the maximum value across all columns
        df[probability_columns] = (df[probability_columns] - df_min) / (df_max - df_min)  # Rescale to [0, 1]

    if apply_lcr_first:  # Apply Luce choice rule if apply_lcr is True
        print(f'Applying lcr FIRST, k = {k}')
        print('BEFORE')
        print(df.tail(5))
        df = lcr(df, k=k)  # Apply LCR to the DataFrame, including only columns in the input
        print(f'AFTER')
        print(df.tail(5))
        print(df.head(5))
        
    if 'Cross' not in df.columns:  # If the 'Cross' column is missing
        sum_columns = ['Target', 'Cohort', 'Rhyme', 'Unrelated']  # Define columns for summation
        sum_columns = [col for col in sum_columns if col in df.columns]  # Filter out columns that are not in the set
        
        if generate_cross:  # If generate_cross is true
            total_sum = df[sum_columns].sum(axis=1)  # Calculate the total sum of probabilities for each row
            trg_max = df['Target'].max()  # Find the maximum value in the 'Target' column
            sum_max = total_sum.max() * sum_weight # Find the maximum value in the 'Target' column
            max_value = sum_max
            if rescale:
                max_value = 1

            df['Cross'] = max_value - (total_sum / sum_max)  # Scale it to decrease gradually
            df['Cross'] = df['Cross'].clip(lower=0)  # Clip the 'Cross' values to avoid negative values
            # # After generating or computing the 'Cross' column, add this code to ensure 'Cross' stays 0 after hitting zero
            cross_zero_mask = df['Cross'].eq(0)  # Identify where 'Cross' reaches zero
            df['Cross'] = df['Cross'].mask(cross_zero_mask.cumsum().gt(0), 0)  # Set 'Cross' to 0 for the rest of the timesteps
        else:  # If generate_cross is false
            df['Cross'] = np.nan  # Set the 'Cross' column to NaN values

    if rescale:  # Apply rescaling if rescale is True
        print('Rescaling data')
        df_min = df[probability_columns].min().min()  # Find the minimum value across all columns
        df_max = df[probability_columns].max().max()  # Find the maximum value across all columns
        df[probability_columns] = (df[probability_columns] - df_min) / (df_max - df_min)  # Rescale to [0, 1]

    df['Time'] = df['Time'] * tmult  # Multiply the time column by tmult factor
    if do_interpolate_missing_time_steps:  # If interpolation is enabled
        print('Interpolating missing time steps')
        df = interpolate_missing_time_steps(df)  # Interpolate missing time steps
        # input_df = df.copy(deep=True)  # Create a deep copy of the DataFrame

    if apply_lcr:  # Apply Luce choice rule if apply_lcr is True
        print(f'Applying lcr, k={k}')
        df = lcr(df, k=k)  # Apply LCR to the DataFrame, including columns in the input AND cross if it's been added

    if normalize:  # Apply normalization if normalize is True
        row_sums = df[probability_columns].sum(axis=1)  # Get the sum for each row
        non_zero_mask = df[probability_columns] != 0  # Create a mask where the values are non-zero
        df[probability_columns] = df[probability_columns].where(~non_zero_mask, df[probability_columns].div(row_sums, axis=0))  # Normalize only non-zero values
    
    if apply_softmax:  # If softmax is enabled
        df = softmax(df[probability_columns].values, axis=1)  # Apply standard softmax transformation
    
    # Trim the DataFrame to only include rows where 'Time' is less than or equal to tmax
    df = df[df['Time'] <= tmax]

    return df  # Return transformed DataFrame

In [2]:
import matplotlib.pyplot as plt
import numpy as np

def calculate_metrics(human_df, comp_df, modelname):
    """
    Calculates objective metrics (RMSE, MAE, Correlation) to measure the difference 
    between human and simulated fixation proportions.
    """
    item_types = ["Target", "Cohort", "Rhyme", "Unrelated", "Cross"]
    
    # Align dataframes by length
    min_len = min(len(human_df), len(comp_df))
    h_df = human_df.iloc[:min_len].reset_index(drop=True)
    c_df = comp_df.iloc[:min_len].reset_index(drop=True)
    
    metrics = {}
    print(f"\n{'='*20} {modelname} Objective Metrics {'='*20}")
    print(f"{'Item':<12} | {'RMSE':<10} | {'MAE':<10} | {'Correlation':<10}")
    print("-" * 55)
    
    total_sq_error = 0
    total_abs_error = 0
    total_count = 0
    
    for item in item_types:
        if item in h_df.columns and item in c_df.columns:
            y_true = h_df[item]
            y_pred = c_df[item]
            
            # RMSE
            mse = np.mean((y_true - y_pred)**2)
            rmse = np.sqrt(mse)
            
            # MAE
            mae = np.mean(np.abs(y_true - y_pred))
            
            # Correlation
            if y_true.std() == 0 or y_pred.std() == 0:
                corr = 0
            else:
                corr = np.corrcoef(y_true, y_pred)[0, 1]
                
            metrics[item] = {'RMSE': rmse, 'MAE': mae, 'Correlation': corr}
            print(f"{item:<12} | {rmse:.4f}     | {mae:.4f}     | {corr:.4f}")
            
            total_sq_error += np.sum((y_true - y_pred)**2)
            total_abs_error += np.sum(np.abs(y_true - y_pred))
            total_count += len(y_true)
            
    # Global weighted metrics
    if total_count > 0:
        global_rmse = np.sqrt(total_sq_error / total_count)
        global_mae = total_abs_error / total_count
        print("-" * 55)
        print(f"{'Overall':<12} | {global_rmse:.4f}     | {global_mae:.4f}     | {'-':<10}")
        print(f"{'='*60}\n")
        
    return metrics

def plot_fixation_proportions(human_df, comp_df, modelname, symbol_interval=25):
    markers = {"Target": "o", "Cohort": "s", "Rhyme": "^", "Unrelated": ".", "Cross": "x"}  # Define markers for different item types
    item_types = ["Target", "Cohort", "Rhyme", "Unrelated", "Cross"]  # List of item types

    # Define custom font sizes
    title_fontsize = 24
    axis_label_fontsize = 22
    tick_labelsize = 18
    legend_fontsize = 18
    annotate_fontsize = 32
    
    # Ensure both dataframes cover the same time range for comparison
    # We assume 'Time' is available or the index represents time. 
    # Based on read_csv, 'Time' is a column but indices are reset.
    # Let's align them based on the minimum common length to simply truncate the longer one,
    # or better, align by their 'Time' column if possible.
    
    # Assuming standard index alignment (0, 1, 2...) if they start at same relative time:
    min_len = min(len(human_df), len(comp_df))
    human_df_clipped = human_df.iloc[:min_len]
    comp_df_clipped = comp_df.iloc[:min_len]
    
    fig, ax = plt.subplots(figsize=(20, 12))
    for item in item_types:
        if item in human_df_clipped.columns and item in comp_df_clipped.columns:
            difference = human_df_clipped[item] - comp_df_clipped[item]
            # Use the time column if available for x-axis, else use index
            if 'Time' in human_df_clipped.columns:
                x_axis = human_df_clipped['Time']
            else:
                x_axis = human_df_clipped.index
                
            ax.plot(x_axis, difference, label=item, marker=markers.get(item, ''), markevery=symbol_interval)

    ax.set_title('Differences (human - simulated)', fontsize=title_fontsize)
    ax.set_xlabel('Time step', fontsize=axis_label_fontsize)
    ax.set_ylabel('Fixation proportion difference', fontsize=axis_label_fontsize)
    ax.tick_params(axis='both', labelsize=tick_labelsize)  # Set tick label size
    ax.legend(fontsize=legend_fontsize)
    ax.set_ylim(-1, 1)
    fig.suptitle(f'Differences (human - simulated) for {modelname}', fontsize=title_fontsize)
    fig.savefig(f'DIFF/differences_{modelname}.png')
    
    # show human df and comp df  
    fig, axes = plt.subplots(2, 1, figsize=(20, 24))
    axes[0].set_title('Human Data', fontsize=title_fontsize)
    axes[1].set_title('Simulated Data', fontsize=title_fontsize)
    for item in item_types:
        if item in human_df_clipped.columns:
            axes[0].plot(human_df_clipped['Time'], human_df_clipped[item], label=item, marker=markers.get(item, ''), markevery=symbol_interval)
        if item in comp_df_clipped.columns:
            # axes[1].axvline(x=480, color='gray', linestyle='--', alpha=0.5, label="theoratical earliest RT")  # Add vertical line at the start of the time range
            axes[1].plot(comp_df_clipped['Time'], comp_df_clipped[item], label=item, marker=markers.get(item, ''), markevery=symbol_interval)
    for ax in axes:
        ax.set_xlabel('Time step', fontsize=axis_label_fontsize)
        ax.set_ylabel('Fixation proportion', fontsize=axis_label_fontsize)
        ax.tick_params(axis='both', labelsize=tick_labelsize)  # Set tick label size
        ax.legend(fontsize=legend_fontsize)
    fig.savefig(f'OUT/fixation_proportions_{modelname}.png')


In [3]:
import os, sys

# Get the current working directory
current_dir = os.getcwd()

# Check if we are in the 'notebooks' directory and define project root accordingly
if os.path.basename(current_dir) == 'notebooks':
    project_root = os.path.dirname(current_dir)
else:
    # Assume we are already at the project root or handle other structures as needed
    project_root = current_dir

if project_root not in sys.path:
    print(f"Adding {project_root} to sys.path")
    sys.path.append(project_root)
    
model2res = {
    # Causal Models from train.sh
    "baseline": "experiments/en_words_ku_baseline/Baseline/training/competition_mean.csv",
    "causal-cnn": "experiments/en_words_ku_causal_cnn/causal-cnn/training/competition_mean.csv",
    "causal-trans": "experiments/en_words_ku_causal_trans/causal-trans/training/competition_mean.csv",
    "causal-rcnn": "experiments/en_words_ku_causal_rcnn/causal-rcnn/training/competition_mean.csv",
    "causal-2lstm": "experiments/en_words_ku_causal_2lstm/causal-2LSTM/training/competition_mean.csv",
    # "causal-ctrans": "experiments/en_words_ku_causal_convtrans/causal-convtrans/training/competition_mean.csv",

    # Non-Causal Models from train.sh
    "noncausal-2lstm": "experiments/en_words_ku_2lstmbi/noncausal-2LSTM/training/competition_mean.csv",
    # "noncausal-convtrans": "experiments/en_words_ku_convtrans/noncausal-convtrans/training/competition_mean.csv",
    "noncausal-rcnn": "experiments/en_words_ku_rcnn/noncausal-rcnn/training/competition_mean.csv",
    "noncausal-cnn": "experiments/en_words_ku_cnn/noncausal-cnn/training/competition_mean.csv",
    "noncausal-trans": "experiments/en_words_ku_trans/noncausal-trans/training/competition_mean.csv"
}
input_path = os.path.join(project_root, "notebooks/INPUT/amt_human_mean.csv")
human_df = read_csv(input_path, normalize=False, rescale=False, tmult=1, 
                    apply_lcr_first=False, apply_lcr=False, k=1, bgate=False, bthresh=0.01, tmax=1000)
    
for modelname, csv_path in model2res.items():
    print(f"Processing csv: {csv_path}")
    # Construct absolute path to ensure we find the file regardless of CWD
    full_csv_path = os.path.join(project_root, csv_path)
 
    comp_df = read_csv(full_csv_path, normalize=False, rescale=False, tmult=10, 
                    apply_lcr_first=False, apply_lcr=False, k=1, bgate=False, bthresh=0.01, tmax=1000)

    # plot_fixation_proportions(
    #     human_df,  # "INPUT/amt_human_mean.csv"
    #     comp_df, # underlying response probabilities
    #     modelname
    # )

    # Calculate and print objective metrics
    metrics = calculate_metrics(human_df, comp_df, modelname) 

Adding /home/fie24002/earshot_nn to sys.path
Interpolating missing time steps
Processing csv: experiments/en_words_ku_baseline/Baseline/training/competition_mean.csv
Interpolating missing time steps

==================== baseline Objective Metrics ====================
Item         | RMSE       | MAE        | Correlation
-------------------------------------------------------
Target       | 0.1287     | 0.1018     | 0.9367
Cohort       | 0.0437     | 0.0342     | 0.7735
Rhyme        | 0.0459     | 0.0369     | 0.6009
Unrelated    | 0.0161     | 0.0109     | -0.5328
Cross        | 0.1283     | 0.1015     | 0.9774
-------------------------------------------------------
Overall      | 0.0864     | 0.0570     | -         

Processing csv: experiments/en_words_ku_causal_cnn/causal-cnn/training/competition_mean.csv
Interpolating missing time steps

==================== causal-cnn Objective Metrics ====================
Item         | RMSE       | MAE        | Correlation
----------------------